To execute this notebook, you need to either
 - Download a pre-trained model
 - Train an example model by excecuting the model_training_tutorial.ipynb notebook

In this tutorial we will learn to:
- Load a previously trained model
- Extract DeepPrint features from fingerprint images
- Evaluate the performance of the extracted fixed-length representations

## Embedding extraction

After training the model, we can extract the DeepPrint features for the fingerprint images. This is done by calling the `extract` method of the `DeepPrintExtractor` class.

In [1]:
import os

from flx.extractor.fixed_length_extractor import get_DeepPrint_Tex, get_DeepPrint_TexMinu, DeepPrintExtractor

# Dimension and number of training subjects must be known to load the pre-trained model

# To load the pre-trained model parameters use num_training_subjects=8000
extractor: DeepPrintExtractor = get_DeepPrint_TexMinu(num_training_subjects=8000, num_dims=256)

# To load the pre-trained model parameters use
MODEL_DIR: str = os.path.abspath("../../") # Path to the directory containing the model parameters
extractor.load_best_model(MODEL_DIR)

Loaded best model from c:\Users\USER1\Documents\2024b\biometrics\firstTool\best_model.pyt


C:\Users\USER1\Documents\2024b\biometrics\firstTool\fixed-length-fingerprint-extractors\flx\models\torch_helpers.py:84: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpo

Now we need to specify the dataset, for which we want to extract the embeddings

In [2]:
import os

from flx.data.dataset import *
from flx.data.image_loader import MCYTOpticalLoader
from flx.data.transformed_image_loader import TransformedImageLoader
from flx.image_processing.binarization import LazilyAllocatedBinarizer
from flx.data.image_helpers import pad_and_resize_to_deepprint_input_size

# NOTE: If this does not work, enter the absolute path to the notebooks/example-dataset directory here! 
DATASET_PATH: str = os.path.abspath(r"C:\Users\USER1\Documents\2024b\biometrics\project\000\L")

# We will use the SFingeLoader to load the images from the dataset
image_loader = TransformedImageLoader(
        images=MCYTOpticalLoader(DATASET_PATH),
        poses=None,
        transforms=[
            LazilyAllocatedBinarizer(5.0),
            pad_and_resize_to_deepprint_input_size,
        ],
    )

image_dataset: Dataset = Dataset(image_loader, image_loader.ids)

# The second value is for the minutiae branch, which we do not have in this example
texture_embeddings, minutia_embeddings = extractor.extract(image_dataset)

Created IdentifierSet with 4 subjects and a total of 20 samples.


100%|██████████| 1/1 [00:21<00:00, 21.76s/it]


## Benchmarking

To evaluate the embeddings, we want to run a benchmark on them. For this, we must first specify the type of benchmark, and which comparisons should be run.

In [3]:
from flx.scripts.generate_benchmarks import create_verification_benchmark

NUM_IMPRESSIONS_PER_SUBJECT = 5
benchmark = create_verification_benchmark(
    subjects=list(range(image_dataset.num_subjects)),
    impressions_per_subject=list(range(NUM_IMPRESSIONS_PER_SUBJECT))
)

100%|██████████| 4/4 [00:00<?, ?it/s]


Now we can run the benchmark. To do this, we must first specify the matcher (in our case cosine similarity of the embeddings)

In [4]:
from flx.benchmarks.matchers import CosineSimilarityMatcher
from flx.data.embedding_loader import EmbeddingLoader

# We concatenate texture and minutia embedding vectors
embeddings = EmbeddingLoader.combine(texture_embeddings, minutia_embeddings)
matcher = CosineSimilarityMatcher(EmbeddingLoader.combine(texture_embeddings, minutia_embeddings))

results = benchmark.run(matcher)

print(f"Equal-Error-Rate: {results.get_equal_error_rate()}")

Created IdentifierSet with 4 subjects and a total of 20 samples.
Created IdentifierSet with 4 subjects and a total of 20 samples.
Created IdentifierSet with 4 subjects and a total of 20 samples.
Created IdentifierSet with 4 subjects and a total of 20 samples.
Created IdentifierSet with 4 subjects and a total of 20 samples.
Created IdentifierSet with 4 subjects and a total of 20 samples.


100%|██████████| 140/140 [00:00<00:00, 34486.55it/s]

Checking for id: Identifier(0, 0)
Available ids: [<flx.data.dataset.Identifier object at 0x0000020065C067D0>, <flx.data.dataset.Identifier object at 0x0000020064B71BD0>, <flx.data.dataset.Identifier object at 0x0000020065C05C10>, <flx.data.dataset.Identifier object at 0x0000020065C05DD0>, <flx.data.dataset.Identifier object at 0x0000020065C05F50>, <flx.data.dataset.Identifier object at 0x0000020065C060D0>, <flx.data.dataset.Identifier object at 0x0000020065C06290>, <flx.data.dataset.Identifier object at 0x0000020065C06450>, <flx.data.dataset.Identifier object at 0x0000020065C06610>, <flx.data.dataset.Identifier object at 0x0000020065C05B50>, <flx.data.dataset.Identifier object at 0x0000020065C05010>, <flx.data.dataset.Identifier object at 0x0000020065C04C90>, <flx.data.dataset.Identifier object at 0x0000020065C051D0>, <flx.data.dataset.Identifier object at 0x0000020065C05390>, <flx.data.dataset.Identifier object at 0x0000020065C05610>, <flx.data.dataset.Identifier object at 0x000002006

To visualize the results, we can plot a DET curve. (Do not wonder if it is empty, probably the model is not trained enough. Take a look at the EER instead.)